In [2]:
import random

In [3]:
def are_valid_clause_variables(clause, num_variables):
    for elem in clause:
        if clause.count(elem) > 1 or (elem in clause and elem + num_variables in clause):
            return False
    return True

In [4]:
def create_clause(list_of_variables, num_variables):
    clause = ""
    random_clause_variables = [0, 0, 0]
    while not are_valid_clause_variables(random_clause_variables, num_variables):
        random_clause_variables = random.sample(list_of_variables, 3)
    
    if random_clause_variables[0] >= num_variables:
        clause += f"n{random_clause_variables[0]-num_variables}"
    else:
        clause += f"{random_clause_variables[0]}"

    if random_clause_variables[1] >= num_variables:
        clause += f" n{random_clause_variables[1]-num_variables}"
    else:
        clause += f" {random_clause_variables[1]}"

    if random_clause_variables[2] >= num_variables:
        clause += f" n{random_clause_variables[2]-num_variables}"
    else:
        clause += f" {random_clause_variables[2]}"
    
    return clause

In [5]:
def exists(clause_to_check, set_to_check):
    if clause_to_check == None:
        return True
    for clause in set_to_check:
        count = 0
        [v1, v2, v3] = clause.split(" ")
        for variable in clause_to_check.split(" "):
            if v1 == variable or v2 == variable or v3 == variable:
                count += 1
        if count == 3:
            return True
    return False

In [6]:
def fac(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * fac(n-1)

In [7]:
def n_choose_k(n, k) -> int:
    return int(fac(n)/(fac(k)*fac(n-k)))

In [8]:
def create_cnf(num_variables, num_clauses, distinct_clauses=False):
    if num_variables < 3:
        print("ERROR: Not possible with num_var < 3")
        return None

    if distinct_clauses and num_clauses > n_choose_k(2*num_variables, 3):
        print("ERROR: Not possible. Decrease the number of clauses or increase the number of variables")
        return None
    cnf = ""
    list_of_variables = list(range(2*num_variables))
    existing_clauses = set()
    for _ in range(num_clauses-1):
        created_clause = None
        if distinct_clauses:
            while created_clause == None or exists(created_clause, existing_clauses):
                created_clause = create_clause(list_of_variables, num_variables)
        else:
            created_clause = create_clause(list_of_variables, num_variables)
        cnf += f"{created_clause},"
        existing_clauses.add(created_clause)
    
    created_clause = None
    if distinct_clauses:
        while created_clause == None or exists(created_clause, existing_clauses):
            created_clause = create_clause(list_of_variables, num_variables)
    else:
        created_clause = create_clause(list_of_variables, num_variables)
    cnf += f"{created_clause}"
    return cnf

In [9]:
def satisfies_cnf_nae(candidate_solution, cnf, num_variables):
    clauses = cnf.split(",")
    for clause in clauses:
        clause_vars = clause.split(" ")
        clause_result = False
        for i in range(len(clause_vars)):
            if clause_vars[i].startswith("n"):
                clause_vars[i] = not candidate_solution[int(clause_vars[i][1:])-num_variables]
            else:
                clause_vars[i] = candidate_solution[int(clause_vars[i])]
            clause_result += clause_vars[i]
        if clause_result == 3 or clause_result == 0:
            return False
    return True


In [10]:
def satisfiable_or_not(num_variables, cnf):
    num_possible_combinations = 2**num_variables
    for i in range(num_possible_combinations):
        binary_string = format(i, 'b').zfill(num_variables)
        candidate_solution = list(map(int, list(binary_string)))
        if satisfies_cnf_nae(candidate_solution, cnf, num_variables):
            return True, candidate_solution
    return False, []

In [12]:
nae3sat_cnfs_file = open("nae3sat_cnfs.txt", "w")
non_nae3sat_cnfs_file = open("non-nae3sat_cnfs.txt", "w")
max_vars = 8
for num_vars in range(3, max_vars):
    for num_clauses in range(num_vars, n_choose_k(2*num_vars, 3)):
        for i in range(20):
            cnf = create_cnf(num_vars, num_clauses, distinct_clauses=False)
            print(f"For {num_vars}/{max_vars} on {num_clauses}/{n_choose_k(2*num_vars, 3)}", end="\r")
            if cnf != None:
                satisfiable, solution = satisfiable_or_not(num_vars, cnf)
                if satisfiable:
                    nae3sat_cnfs_file.write(f"{num_vars} -- {cnf}\n")
                else:
                    non_nae3sat_cnfs_file.write(f"{num_vars} -- {cnf}\n")
nae3sat_cnfs_file.close()
non_nae3sat_cnfs_file.close()
